In [1]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch

import os
import pandas as pd
import geopandas as gpd
import xml.etree.ElementTree as ET
import gzip
import json
import fiona
from collections import defaultdict

from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch

highway_mapping = {
    'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
    'primary': 1, 'primary_link': 1,
    'secondary': 2, 'secondary_link': 2,
    'tertiary': 3, 'tertiary_link': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'np.nan': -1
}

base_dir = '../../../../data/pop_1pm_simulations/idf_1pm/' 

In [2]:
# List all the files in the base_dir
files = os.listdir(base_dir)

for file in files:
    file_path = os.path.join(base_dir, file)
    base_name, ext = os.path.splitext(file)
    if base_name.startswith("idf_1pm_"):
        base_name = base_name.replace("idf_1pm_", "")
    var_name = base_name  # Start with the cleaned base name
    
    if file.endswith('.csv'):
        try:
            var_name = f"{var_name}_df"  # Add "_c" for CSV files
            globals()[var_name] = pd.read_csv(file_path, sep=";")
            print(f"Loaded CSV file: {file} into variable: {var_name}")
        except Exception as e:
            print(f"Error loading CSV file {file}: {e}")

Loaded CSV file: idf_1pm_persons.csv into variable: persons_df
Loaded CSV file: idf_1pm_households.csv into variable: households_df
Loaded CSV file: idf_1pm_trips.csv into variable: trips_df
Loaded CSV file: idf_1pm_activities.csv into variable: activities_df
Loaded CSV file: idf_1pm_vehicle_types.csv into variable: vehicle_types_df
Loaded CSV file: idf_1pm_vehicles.csv into variable: vehicles_df


# Relevant dataframes are:

activities_g
commutes_g
households_c
trips_g
vehicles_c

In [3]:
population_df = pd.read_csv("intermediate_results/population.csv", sep=";")
transit_schedule_relations_df = pd.read_csv("intermediate_results/transit_schedule_relations.csv", sep=";")
transit_schedule_stop_facilities_df = pd.read_csv("intermediate_results/transit_schedule_stop_facilities.csv", sep=";")